In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
from scipy.spatial.distance import pdist
from scipy.spatial import cKDTree
from sklearn.neighbors import BallTree
from pyproj import Geod 
from geopy.distance import geodesic

from collections import Counter
from tqdm import tqdm
from shapely import wkt
from shapely import wkb

geod = Geod(ellps="WGS84")

In [2]:
# Constants
EARTH_RADIUS = 6371  # Earth radius in km

In [ ]:
#input is an output from preprocessing2_building_density
df_K = pd.read_parquet(r"TN_residential_A_PM_BD.parquet")
df_K["geometry"] = df_K["geometry"].apply(wkb.loads)
df_K = gpd.GeoDataFrame(df_K, geometry='geometry')


#df_K = gpd.GeoDataFrame(df_K, geometry=shapely.from_wkb(df_K['geometry']))
df_K.index = [i for i in range(len(df_K))]



### Perimiter to area ratio

In [ ]:
df_K.head()


In [ ]:

df_K['building_perimeter_in_meters_new'] = df_K["geometry"].apply(lambda g: geod.geometry_area_perimeter(g)[1]) 

In [ ]:
df_K['perimeter_to_area_ratio'] = df_K['building_perimeter_in_meters_new'] / df_K['area_in_meters']
df_K

In [ ]:
df_K['perimeter_to_area_ratio'] = df_K['perimeter_to_area_ratio'].clip(upper=6.5)
print(df_K['perimeter_to_area_ratio'].describe())

In [ ]:
df_K['perimeter_to_area_ratio'].max()

In [12]:
df_K['normalized_perimeter_to_area_ratio'] = df_K['perimeter_to_area_ratio'] / df_K['perimeter_to_area_ratio'].max()

### Radius calculation

df_K['geo_original']=df_K['geometry']

In [ ]:
df_K.set_crs("EPSG:4326", inplace=True, allow_override=True)  
print("Current CRS:", df_K.crs)

# Convert to a projected CRS for Kenya (EPSG:32736 - UTM Zone 36S)
#df_K = df_K.to_crs("EPSG:32736")  
df_K = df_K.to_crs("EPSG:7767") 

# Now calculate the centroid correctly
df_K["centroid"] = df_K.geometry.centroid

print(df_K[["geometry", "centroid"]].head())


In [ ]:

def calculate_radius(geometry):
    
    geometry = geometry if geometry.type == 'Polygon' else geometry.convex_hull
    centroid = geometry.centroid
    boundary_points = np.array(geometry.exterior.coords)  # Get boundary coordinates
    distances = np.linalg.norm(boundary_points - np.array([centroid.x, centroid.y]), axis=1)  # Compute distances
    return np.mean(distances)  # Take the mean distance


# Compute radius in meters
df_K["radius_m"] = df_K["geometry"].apply(calculate_radius)

# Convert back to WGS84 (if needed)
df_K = df_K.to_crs("EPSG:4326")

print(df_K[["centroid", "radius_m"]].head())


In [ ]:
df_K["num_vertices"] = df_K["geometry"].apply(lambda x: len(x.exterior.coords) if x.type == 'Polygon' else sum(len(g.exterior.coords) for g in x.geoms))
# df_K["num_vertices"] = df_K["geometry"].apply(lambda x: len(x.exterior.coords))
df_K[["num_vertices"]].describe()

### Roads calculation

In [ ]:

#this file is an output from helper_extract_roads
mroads=gpd.read_file(r'osm_highways_inside_Tamil_Nadu.geojson')
mroads.set_crs(epsg=4326)
mroads.to_parquet('Tamil_Nadu_Roads.parquet')

final_gdf = gpd.read_parquet('Tamil_Nadu_Roads.parquet')
final_gdf = final_gdf.set_crs("EPSG:4326")
final_gdf['road_index'] = [i for i in range(len(final_gdf))]

In [ ]:
final_gdf

In [20]:
len(list(final_gdf.columns))

395

In [ ]:
print(final_gdf.geometry.type.value_counts())
print("Empty geometries count:", final_gdf.geometry.is_empty.sum())
print("Unique highway values:", final_gdf['highway'].unique())

In [22]:
required_columns = {"highway", "geometry", "id",'width','oneway','junction','lanes','maxspeed','motorcar', 'road_index'}
final_gdf = final_gdf[[col for col in required_columns if col in final_gdf.columns]]

final_gdf

,width,lanes,geometry,junction,maxspeed,motorcar,road_index,oneway,id,highway
0,None,None,POINT (80.26517 13.08078),None,None,None,0,None,30037235,traffic_signals
1,None,None,POINT (80.26278 13.0803),None,None,None,1,None,30037236,traffic_signals
2,None,None,POINT (80.25877 13.07943),None,None,None,2,None,30037239,traffic_signals
3,None,None,POINT (80.25468 13.079),None,None,None,3,None,30037241,traffic_signals
4,None,None,POINT (80.25169 13.07889),None,None,None,4,None,30037243,traffic_signals
...,...,...,...,...,...,...,...,...,...,...
967517,None,None,"LINESTRING (78.50644 12.10761, 78.50659 12.107...",None,None,None,967517,None,1394929912,unclassified
967518,None,None,"LINESTRING (78.50638 12.10832, 78.50675 12.108...",None,None,None,967518,None,1394929913,unclassified
967519,None,None,"LINESTRING (78.50653 12.10866, 78.50681 12.10861)",None,None,None,967519,None,1394929914,unclassified
967520,None,None,"LINESTRING (78.50679 12.10841, 78.50699 12.108...",None,None,None,967520,None,1394929915,unclassified


In [23]:
duplic=final_gdf[final_gdf.duplicated(keep=False)]
print(duplic.shape)

(0, 10)


In [ ]:
tolerance = 0.00001 

final_gdf['geometry_simplified'] = final_gdf['geometry'].simplify(tolerance)
final_gdf

In [ ]:
final_gdf

In [26]:
roads_categories = {
    1: ['motorway', 'trunk_link', 'motorway_link', 'trunk', 'primary', 'primary_link'],
    2: ['secondary', 'secondary_link',],
    3: ['tertiary', 'tertiary_link', ],
    4: ['residential', 'footway', 'service', 'unclassified','living_street','steps','path','track','pedestrian','cycleway','raceway','bridleway','construction','services','bus_stop','road','rest_area','yes','emergency_access_point','corridor','junction','proposed','minor']
    }

In [ ]:
def explode_multilinestrings(gdf):
    """ Convert MultiLineStrings to separate LineStrings """
    gdf = gdf.explode(ignore_index=True) 
    return gdf[gdf.geometry.type == 'LineString']

final_gdf = final_gdf[final_gdf.geometry.notnull()]
final_gdf = explode_multilinestrings(final_gdf)
final_gdf

In [28]:

# Convert roads to a projected CRS (Web Mercator, meters) for accurate centroid calculation
projected_crs = "EPSG:3857"
final_gdf = final_gdf.to_crs(projected_crs)
df_K = df_K.to_crs("EPSG:3857")


In [29]:
df_K['centroid_x'] = df_K.geometry.apply(lambda g: g.centroid.xy[0][0])
df_K['centroid_y'] = df_K.geometry.apply(lambda g: g.centroid.xy[1][0])


In [ ]:
    
# Function to calculate geodesic distance (true Earth distance in meters)
def geodesic_distance(house_row, road_row):
    house_coords = (house_row['latitude'], house_row['longitude'])  
    road_coords = (road_row.geometry_centroid.y, road_row.geometry_centroid.x)  
    return geodesic(house_coords, road_coords).meters  

In [31]:
def explode_road_geometry(df):
    
    road_rows = []
    for row_idx, row in df.to_dict(orient='index').items():
        
        for x, y in row['geometry'].coords:
            current_row = row.copy()
            current_row['geometry_centroid'] = shapely.Point(x, y)
            current_row['row_idx'] = row_idx
            road_rows.append(current_row)

    result_df = pd.DataFrame.from_dict(road_rows)
    result_df.index = [i for i in range(len(result_df))]    
    
    result_df['centroid_x'] = result_df.geometry.apply(lambda g: g.centroid.xy[0][0])
    result_df['centroid_y'] = result_df.geometry.apply(lambda g: g.centroid.xy[1][0]) 
    
    return result_df

In [32]:
category_bbox_size = {
    1: 5_000,
    2: 4_000, 
    3: 3_000,
    4: 2_000
}

In [33]:
df_K = df_K.sort_values(by='area_in_meters', ascending=True)
df_K.index = [i for i in range(len(df_K))]
df_K

,id,geometry,bbox,version,sources,level,building_density_50,building_density_100,building_density_250,building_density_500,...,SMOD_name,SMOD_id,building_perimeter_in_meters_new,perimeter_to_area_ratio,normalized_perimeter_to_area_ratio,centroid,radius_m,num_vertices,centroid_x,centroid_y
0,78.97214059081223:10.135088002360861,"POLYGON ((8791139.714 1134164.24, 8791137.488 ...","{'xmax': 78.97215270996094, 'xmin': 78.9721221...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,29,43,117,301,...,LOW_DENSITY_RURAL,1,9.228217,1.769546,1.000000,POINT (1246135.368 30976.493),1.652855,5,8.791138e+06,1.134163e+06
1,78.85824744438294:10.317846475704181,"POLYGON ((8778458.832 1154834.535, 8778460.947...","{'xmax': 78.85826110839844, 'xmin': 78.8582305...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,56,104,153,333,...,SUBURBAN_PERI_URBAN,3,10.375396,1.604408,0.906678,POINT (1233277.4 51247.31),1.888744,5,8.778460e+06,1.154836e+06
2,79.10840261337607:10.880464721230062,"POLYGON ((8806307.585 1218555.689, 8806305.269...","{'xmax': 79.1084213256836, 'xmin': 79.10838317...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,57,186,695,1547,...,URBAN_CENTER,6,10.387247,1.540844,0.870757,POINT (1260118.716 114416.215),1.851703,5,8.806307e+06,1.218554e+06
3,79.15298885000001:10.822422004291047,"POLYGON ((8811272.056 1211976.107, 8811268.772...","{'xmax': 79.15301513671875, 'xmin': 79.1529617...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,19,43,125,255,...,LOW_DENSITY_RURAL,1,10.566380,1.544801,0.872993,POINT (1265123.938 108005.419),1.912442,5,8.811270e+06,1.211975e+06
4,79.10889275156667:10.881961082418233,"POLYGON ((8806361.864 1218725.612, 8806359.838...","{'xmax': 79.1089096069336, 'xmin': 79.10886383...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,27,133,639,1458,...,URBAN_CENTER,6,11.443114,1.470302,0.830892,POINT (1260170.548 114583.902),2.089631,5,8.806362e+06,1.218724e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4611,78.96298986117297:11.741805696545807,"POLYGON ((8790191.261 1316359.803, 8790065.603...","{'xmax': 78.96363830566406, 'xmin': 78.9623565...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,6,60,423,1535,...,URBAN_CENTER,6,391.548177,0.044139,0.024944,POINT (1242881.226 210227.808),72.477349,5,8.790120e+06,1.316338e+06
4612,79.21022519527399:10.569520347921756,"POLYGON ((8817538.463 1183329.5, 8817556.13 11...","{'xmax': 79.21089935302734, 'xmin': 79.2092895...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,42,58,64,93,...,LOW_DENSITY_RURAL,1,584.961140,0.031992,0.018079,POINT (1271830.761 79860.954),90.986627,7,8.817642e+06,1.183324e+06
4613,79.20410278150487:10.568647243131831,"POLYGON ((8816954.782 1183263.707, 8816821.688...","{'xmax': 79.20545959472656, 'xmin': 79.2028045...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,26,92,222,288,...,LOW_DENSITY_RURAL,1,704.719307,0.036015,0.020353,POINT (1271155.689 79754.073),104.511378,7,8.816960e+06,1.183225e+06
4614,79.2296785307081:10.045556391696277,"POLYGON ((8819884.421 1124228.875, 8819859.308...","{'xmax': 79.23037719726562, 'xmin': 79.2288970...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,21,85,295,959,...,SUBURBAN_PERI_URBAN,3,947.111414,0.024548,0.013873,POINT (1274799.97 21346.014),144.226629,14,8.819807e+06,1.124040e+06


In [ ]:
for category, road_types in roads_categories.items():

    print(f'Processing road_types: {road_types}')
    
    filtered_roads_df = final_gdf[final_gdf['highway'].isin(road_types)]
    print(f'Unexploded road geometries amount: {len(filtered_roads_df)}')
    
    filtered_roads_df = explode_road_geometry(filtered_roads_df)
    
    print(f'Exploded road geometries amount: {len(filtered_roads_df)}')
    road_centroids = filtered_roads_df['geometry_centroid']  

    road_coords = np.array([(point.x, point.y) for point in road_centroids if not point.is_empty])
    
    if len(road_coords) == 0:
        raise ValueError("No valid road centroids found. Check road geometries!")
    
    road_tree = cKDTree(road_coords)

    house_coords = np.array(list(zip(df_K.centroid_x, df_K.centroid_y)))  

    distances, indices = road_tree.query(house_coords, k=1)
    
    distance_col_name = f'distance_to_{category}'
    road_type_col_name = f'nearest_road_type_{category}'
    
    df_K[road_type_col_name] = ''
    for building_idx, (distance, idx) in tqdm(enumerate(zip(distances, indices)), desc='Assigning roads & distances', total=len(distances)):
        
        df_K.loc[building_idx, distance_col_name] = float(distance)
        df_K.loc[building_idx, road_type_col_name] = filtered_roads_df.iloc[idx].highway
        
    

    

In [ ]:
distance_columns = ['distance_to_1', 'distance_to_2', 'distance_to_3', 'distance_to_4']

df_K[distance_columns].describe()

In [36]:
for cat, limit in category_bbox_size.items():
    
    df_K[f'distance_to_{cat}'] = df_K[f'distance_to_{cat}'].clip(lower=0, upper=limit)

In [ ]:
distance_columns = ['distance_to_1', 'distance_to_2', 'distance_to_3', 'distance_to_4']

df_K[distance_columns].describe()

In [ ]:
df_K.columns

In [39]:
fixed_radius_by_category = {
    1: 500,
    2: 400,
    3: 300,
    4: 200,
}

### Density of roads

In [41]:
def explode_road_geometry_without_index(df):
    
    road_rows = []
    for row_idx, row in df.to_dict(orient='index').items():
        
        for x, y in row['geometry'].coords:
            current_row = row.copy()
            current_row['geometry_centroid'] = shapely.Point(x, y)
            current_row['row_idx'] = row_idx
            road_rows.append(current_row)

    result_df = pd.DataFrame.from_dict(road_rows)  
    
    result_df['centroid_x'] = result_df.geometry.apply(lambda g: g.centroid.xy[0][0])
    result_df['centroid_y'] = result_df.geometry.apply(lambda g: g.centroid.xy[1][0]) 
    
    return result_df

In [42]:
filtered_roads = explode_road_geometry_without_index(final_gdf)

In [43]:
road_centroids = filtered_roads['geometry_centroid'] 
road_coords = np.array([(point.x, point.y) for point in road_centroids if not point.is_empty])
   
road_tree = cKDTree(road_coords)

In [44]:
import warnings
warnings.filterwarnings("ignore")

In [45]:
fixed_radius_by_category = {
    1: 500,
    2: 400,
    3: 300,
    4: 200,
    5: 100,
}

In [46]:
roads_categories_ = {
    4: ['residential', 'footway', 'service', 'unclassified','living_street','steps','path','track','pedestrian','cycleway','raceway','bridleway','construction','services','bus_stop','road','rest_area','yes','emergency_access_point','corridor','junction','proposed','minor'],
    5: ['residential', 'footway', 'service', 'unclassified','living_street','steps','path','track','pedestrian','cycleway','raceway','bridleway','construction','services','bus_stop','road','rest_area','yes','emergency_access_point','corridor','junction','proposed','minor']
    }

In [47]:
def compute_road_density(road_tree, building_x, building_y, radius, category_roads_df):
    # Get nearby roads
    nearby_indices = road_tree.query_ball_point((building_x, building_y), radius)
    
    building_radius_polygon = shapely.Point(building_x, building_y).buffer(radius)
    
    if not nearby_indices:
        return 0  

    # Get total road length within radius
    
    road_idxs = list(set(filtered_roads.loc[nearby_indices].row_idx))
    
    filtered_roads_df = category_roads_df[category_roads_df.road_index.isin(road_idxs)].copy()
    
    filtered_roads_df['geometry'] = filtered_roads_df['geometry'].apply(lambda g: g.intersection(building_radius_polygon))
    total_road_length = filtered_roads_df.geometry.length.sum()
    
    # Compute buffer area
    buffer_area = np.pi * (radius ** 2)  # Circle area formula πr²
    
    # Compute density: road length per km²
    return (total_road_length / buffer_area) * 1e6  # Convert to km/km²

building_coords = np.array(list(zip(df_K.centroid_x, df_K.centroid_y)))

for category, road_types in roads_categories_.items():

    fixed_radius = fixed_radius_by_category[category]
    
    category_roads_df = final_gdf[final_gdf.highway.isin(road_types)]
    # df_K[f"road_density_for_{category}_fixed"] = df_K[['centroid_x', 'centroid_y']].apply(lambda row: compute_road_density(row.centroid_x, row.centroid_y, fixed_radius), axis=1)
    # [ for x, y in tqdm(building_coords, total=len(building_coords))]
#  Compute road density for all buildings (Vectorized, fast)
    df_K[f"road_density_for_{category}_fixed"] = [compute_road_density(road_tree, x, y, fixed_radius, category_roads_df) for x, y in tqdm(building_coords, total=len(building_coords), desc=f'Counting for fixed radius: {fixed_radius}')]

Counting for fixed radius: 100: 100%|██████████| 4616/4616 [00:50<00:00, 92.27it/s] 


In [48]:
density_columns = ['road_density_for_4_fixed', 'road_density_for_5_fixed']

df_K[density_columns].describe()

,road_density_for_4_fixed,road_density_for_5_fixed
count,4616.000000,4616.000000
mean,12065.357251,14603.890418
std,7066.767067,8013.743428
min,0.000000,0.000000
25%,6891.572875,8171.088384
50%,10568.291251,14544.991449
75%,17296.119501,20999.491099
max,40750.032475,44663.470500


In [ ]:
df_K.to_parquet(r'TN_residential_A_PM_BD_ROADS.parquet')